In [1]:
!pip install graphviz pydot tqdm 
!pip install --user wandb -qqq
#!pip install -Iv protobuf==3.12.0
#!pip install --upgrade tensorflow

In [2]:
import pickle
import imageio
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import save_model
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical 
from graphviz import Digraph
import pydot
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm
import math
import numpy as np
import os
from os import listdir
import wandb

In [3]:
wandb.login()

wandb: Currently logged in as: dolphin_project (use `wandb login --relogin` to force relogin)


True

In [4]:
# Constants
BR = 0.2
BATCH_SIZE = 16
ROT, SCALE = 10, 0.1
NUM_CLASSES = 26
DESIRED_SIZE = (218, 145)
TRAIN_DIR = "I:/University/Courses/Machine Learning/dolphin_dataset/"
LR = 0.001

wandb.init(project="dolphin_project ",
           config={
               "batch_size": BATCH_SIZE,
               "lr" : LR,
               "input_size": DESIRED_SIZE,
               "dataset": "dolphin",
           })

C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Adjust CSV

In [5]:
#adjust names to fit
train_csv = TRAIN_DIR + "train.csv"
train_df = pd.read_csv(train_csv)
train_df.species.replace({"globis": "short_finned_pilot_whale",
                          "pilot_whale": "short_finned_pilot_whale",
                          "kiler_whale": "killer_whale",
                          "bottlenose_dolpin": "bottlenose_dolphin"}, inplace=True)

species_labels = list(train_df.species.unique())
images = train_df['image']
sid = train_df['individual_id']
train_df

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392
...,...,...,...
51028,fff639a7a78b3f.jpg,beluga,5ac053677ed1
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9
51031,fffbc5dd642d8c.jpg,beluga,4000b3d7c24e


In [6]:
train_df.species.unique()
len(train_df.species.unique())

26

In [7]:
def get_id(sp):
    return species_labels.index(sp)
##encode species
train_df["species"] = train_df.apply(lambda row :get_id(row["species"]),axis = 1)

##one-hot encode species
#train_df = pd.concat([train_df, pd.get_dummies(train_df["species"],prefix='species_',drop_first=True)], axis = 1)
#train_df.drop(['species'],axis=1, inplace=True)
train_df

,image,species,individual_id
0,00021adfb725ed.jpg,0,cadddb1636b9
1,000562241d384d.jpg,1,1a71fbb72250
2,0007c33415ce37.jpg,2,60008f293a2b
3,0007d9bca26a99.jpg,3,4b00fe572063
4,00087baf5cef7a.jpg,1,8e5253662392
...,...,...,...
51028,fff639a7a78b3f.jpg,4,5ac053677ed1
51029,fff8b32daff17e.jpg,17,1184686361b3
51030,fff94675cc1aef.jpg,7,5401612696b9
51031,fffbc5dd642d8c.jpg,4,4000b3d7c24e


## Load images

In [8]:
def scale_and_rotate_image(im, sx, sy, deg_ccw):
    im_orig = im
    im = Image.new('RGBA', im_orig.size, (255, 255, 255, 255))
    im.paste(im_orig)

    w, h = im.size
    angle = math.radians(-deg_ccw)

    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)

    scaled_w, scaled_h = w * sx, h * sy

    new_w = int(math.ceil(math.fabs(cos_theta * scaled_w) + math.fabs(sin_theta * scaled_h)))
    new_h = int(math.ceil(math.fabs(sin_theta * scaled_w) + math.fabs(cos_theta * scaled_h)))

    cx = w / 2.
    cy = h / 2.
    tx = new_w / 2.
    ty = new_h / 2.

    a = cos_theta / sx
    b = sin_theta / sx
    c = cx - tx * a - ty * b
    d = -sin_theta / sy
    e = cos_theta / sy
    f = cy - tx * d - ty * e

    return im.transform(
        (new_w, new_h),
        Image.AFFINE,
        (a, b, c, d, e, f),
        resample=Image.BILINEAR
    )            

def resize_with_crop_or_pad(im, process=False, flip=False, rotate_scale=None, br=None, non_square=None, crop=None):

    old_size = im.size  # old_size[0] is in (width, height) format
    max_dim = np.argmax(old_size)
    ratio = float(DESIRED_SIZE[max_dim]) / old_size[max_dim]

    #ratio = float(max(desired_size)) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    # use thumbnail() or resize() method to resize the input image

    # thumbnail is a in-place operation

    #im.thumbnail(new_size, Image.ANTIALIAS)
    #im = im.resize(new_size, Image.ANTIALIAS)
    #im = im.convert('RGB')
 
    if crop is not None:
        crop_value = 0.0
        prob = np.random.uniform(0, 1)
        if prob > 0.75:
            crop_value = crop
            im = ImageOps.crop(im, int(crop_value*im.size[1]))
        elif prob > 0.5 and prob <= 0.75:
            crop_value = crop*0.625
        else:
            crop_value = np.abs(np.random.uniform(0, 0.075))

        im = ImageOps.crop(im, int(crop_value*im.size[1]))
    '''
    if rotate_scale is not None:
        sx, sy = np.random.normal(scale=rotate_scale[1])+1, np.random.normal(scale=rotate_scale[1])+1
        r = np.random.normal(scale=rotate_scale[0])
        im = scale_and_rotate_image(im, sx, sy, r)
    ''' 

    if br is not None:
        b = np.random.normal(scale=br)+0.9
        c = np.random.normal(scale=br)+0.9
      
        enhancerc = ImageEnhance.Contrast(im)
        im = enhancerc.enhance(c)
        enhancerb = ImageEnhance.Brightness(im)
        im = enhancerb.enhance(b)
        #im = im.resize((mobilenet_input_shape[0], mobilenet_input_shape[1]), Image.ANTIALIAS)

    im = im.resize(new_size, Image.ANTIALIAS)

    if flip:
        ran = np.random.random_sample()

        if ran >= 0.5:
            im = im.transpose(Image.FLIP_LEFT_RIGHT)
    #im.show()
    # create a new image and paste the resized on it
    if non_square is not None:
        new_im = Image.new("RGB", (DESIRED_SIZE[0], DESIRED_SIZE[1]))
        new_im.paste(im, ((DESIRED_SIZE[0] - new_size[0]) // 2,
                        (DESIRED_SIZE[1] - new_size[1]) // 2))
        if process:
            new_im = np.array(new_im, dtype=np.float32) / 255.
        else:
            new_im = np.array(new_im, dtype=np.float32)
        
        return new_im[:,:,:3]
    
    if process:
        im = np.array(new_im, dtype=np.float32) / 255.
    else:
        im = np.array(im, dtype=np.float32)
    
    return im[:,:,:3]

def augment(im, br=None, crop=None):
    if crop is not None:
        crop_value = 0.0
        prob = np.random.uniform(0, 1)
        if prob > 0.75:
            crop_value = crop
            im = ImageOps.crop(im, int(crop_value*im.size[1]))
        elif prob > 0.5 and prob <= 0.75:
            crop_value = crop*0.625
        else:
            crop_value = np.abs(np.random.uniform(0, 0.075))

        im = ImageOps.crop(im, int(crop_value*im.size[1]))
        im = im.resize(DESIRED_SIZE, Image.ANTIALIAS)
    
    if br is not None:
        b = np.random.normal(scale=br)+0.9
        c = np.random.normal(scale=br)+0.9
      
        enhancerc = ImageEnhance.Contrast(im)
        im = enhancerc.enhance(c)
        enhancerb = ImageEnhance.Brightness(im)
        im = enhancerb.enhance(b)
    
    im = np.array(im, dtype=np.float32)
    return im[:,:,:3]

In [9]:
'''
# resizing dataset --- Create a folder train_images_sized and run this code one time to create the rescaled dataset
# WARNING THIS WILL START MODIFYING THE IMAGES IN "train_images_sized/"
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def resize_with_crop_or_pad_for_resizing(im, process=False, flip=False, rotate_scale=None, br=None, non_square=None, crop=None):

    old_size = im.size  # old_size[0] is in (width, height) format
    max_dim = np.argmax(old_size)
    ratio = float(DESIRED_SIZE[max_dim]) / old_size[max_dim]

    #ratio = float(max(desired_size)) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    # use thumbnail() or resize() method to resize the input image

    # thumbnail is a in-place operation

    #im.thumbnail(new_size, Image.ANTIALIAS)
    #im = im.resize(new_size, Image.ANTIALIAS)
    #im = im.convert('RGB')
 
    if crop is not None:
        crop_value = 0.0
        prob = np.random.uniform(0, 1)
        if prob > 0.75:
            crop_value = crop
            im = ImageOps.crop(im, int(crop_value*im.size[1]))
        elif prob > 0.5 and prob <= 0.75:
            crop_value = crop*0.625
        else:
            crop_value = np.abs(np.random.uniform(0, 0.075))

        im = ImageOps.crop(im, int(crop_value*im.size[1]))
    
    
    if rotate_scale is not None:
        sx, sy = np.random.normal(scale=rotate_scale[1])+1, np.random.normal(scale=rotate_scale[1])+1
        r = np.random.normal(scale=rotate_scale[0])
        im = scale_and_rotate_image(im, sx, sy, r)
    

    if br is not None:
        b = np.random.normal(scale=br)+0.9
        c = np.random.normal(scale=br)+0.9
      
        enhancerc = ImageEnhance.Contrast(im)
        im = enhancerc.enhance(c)
        enhancerb = ImageEnhance.Brightness(im)
        im = enhancerb.enhance(b)
        #im = im.resize((mobilenet_input_shape[0], mobilenet_input_shape[1]), Image.ANTIALIAS)

    im = im.resize(new_size, Image.ANTIALIAS)
    
    
    if flip:
        ran = np.random.random_sample()

        if ran >= 0.5:
            im = im.transpose(Image.FLIP_LEFT_RIGHT)
    
    #im.show()
    # create a new image and paste the resized on it
    if non_square is not None:
        new_im = Image.new("RGB", (DESIRED_SIZE[0], DESIRED_SIZE[1]))
        new_im.paste(im, ((DESIRED_SIZE[0] - new_size[0]) // 2,
                        (DESIRED_SIZE[1] - new_size[1]) // 2))
        #if process:
        #    new_im = np.array(new_im, dtype=np.float32) / 255.
        #else:
        #    new_im = np.array(new_im, dtype=np.float32)
        
        return new_im#[:,:,:3]
    
    if process:
        im = np.array(new_im, dtype=np.float32) / 255.
    else:
        im = np.array(im, dtype=np.float32)
    
    return im[:,:,:3]

for i in range(len(train_df)):
    if os.path.isfile(TRAIN_DIR + "train_images_sized/" + train_df.iat[i, 0]):
        continue
    img = resize_with_crop_or_pad_for_resizing(
                Image.open(TRAIN_DIR + "train_images/" + train_df.iat[i, 0]),
                #process=True,
                #flip=True,
                #br=BR,
                #rotate_scale=(ROT, SCALE),
                non_square=True
            )
    img.save(TRAIN_DIR + "train_images_sized/" + train_df.iat[i, 0])

'''

'\n# resizing dataset --- Create a folder train_images_sized and run this code one time to create the rescaled dataset\n# WARNING THIS WILL START MODIFYING THE IMAGES IN "train_images_sized/"\nfrom PIL import ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES = True\n\ndef resize_with_crop_or_pad_for_resizing(im, process=False, flip=False, rotate_scale=None, br=None, non_square=None, crop=None):\n\n    old_size = im.size  # old_size[0] is in (width, height) format\n    max_dim = np.argmax(old_size)\n    ratio = float(DESIRED_SIZE[max_dim]) / old_size[max_dim]\n\n    #ratio = float(max(desired_size)) / max(old_size)\n    new_size = tuple([int(x * ratio) for x in old_size])\n    # use thumbnail() or resize() method to resize the input image\n\n    # thumbnail is a in-place operation\n\n    #im.thumbnail(new_size, Image.ANTIALIAS)\n    #im = im.resize(new_size, Image.ANTIALIAS)\n    #im = im.convert(\'RGB\')\n \n    if crop is not None:\n        crop_value = 0.0\n        prob = np.random.uniform(

In [10]:
#Training Data
train_df_im_labels = train_df[["image", "species"]]
train_df_im_labels = train_df_im_labels.sample(frac=1, random_state=113)
X_train, X_valid = train_df_im_labels.loc[:len(train_df_im_labels)*9//10], train_df_im_labels.loc[len(train_df_im_labels)*9//10:]
print(len(X_train), len(X_valid))
'''
for i,img in enumerate(tqdm(images)):
    image = cv2.imread("train_images/"+img,cv2.IMREAD_GRAYSCALE)#imports pictures in grayscale since colors have own dimension
    image = cv2.resize(image, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
    #dataset.append((image,sid[i]))
    dataset.append(image)
'''

45732 5302


'\nfor i,img in enumerate(tqdm(images)):\n    image = cv2.imread("train_images/"+img,cv2.IMREAD_GRAYSCALE)#imports pictures in grayscale since colors have own dimension\n    image = cv2.resize(image, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)\n    #dataset.append((image,sid[i]))\n    dataset.append(image)\n'

In [11]:
'''
#Testing Data
test_dir = "test_images"
test_dataset = []
for img in tqdm(os.listdir(test_dir)): 
    image = imageio.imread("test_images/"+img)
    image = cv2.resize(image, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
    test_dataset.append(image)
'''

'\n#Testing Data\ntest_dir = "test_images"\ntest_dataset = []\nfor img in tqdm(os.listdir(test_dir)): \n    image = imageio.imread("test_images/"+img)\n    image = cv2.resize(image, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)\n    test_dataset.append(image)\n'

# Train on batch

In [12]:
def train_on_batch(model, batch_idx):
    batch = []
    y = []
    for idx in batch_idx:
        img = augment(
                Image.open(TRAIN_DIR + "train_images_sized/" + train_df_im_labels.iat[idx, 0]),
                br=BR,
                crop=True,
                #rotate_scale=(ROT, SCALE),
                #non_square=True
            )
        #print(img.shape)
        batch.append(
            img
        )
        y.append(train_df_im_labels.iat[idx, 1])
    
    batch = np.array(batch)
    #print(batch.shape)
    y = to_categorical(y, NUM_CLASSES)
    loss, acc = model.train_on_batch(batch, y)
    wandb.log({"loss": loss, "accuracy": acc})
    return loss, acc

# Validation

In [13]:
def validate(model, X_valid):
    loss, acc = [], []
    bs = 32
    for b in range(len(math.ceil(X_valid/bs))):
        for idx in range(bs):
            img = resize_with_crop_or_pad(
                    Image.open(TRAIN_DIR + "train_images_sized/" + X_valid.iat[b*bs+idx, 0]),
                    #process=True,
                    #flip=True,
                    #br=BR,
                    #rotate_scale=(ROT, SCALE),
                    non_square=True
                )
            #print(img.shape)
            batch.append(
                img
            )
            y.append(train_df_im_labels.iat[b*bs+idx, 1])
    
        batch = np.array(batch)
    #print(batch.shape)
        y = to_categorical(y, NUM_CLASSES)
        l, a = model.train_on_batch(batch, y)
        loss.append(l)
        acc.append(a)
    return np.mean(loss), np.mean(acc)

## Modelling

In [14]:
#generating model
'''
model = keras.Sequential([
    layers.Flatten(input_shape=[64, 64]),
    layers.Dense(512, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(26, activation="softmax"),
])
model.summary()
plot_model(model,show_shapes=True, show_layer_names=True)
'''

'\nmodel = keras.Sequential([\n    layers.Flatten(input_shape=[64, 64]),\n    layers.Dense(512, activation="relu"),\n    layers.Dense(256, activation="relu"),\n    layers.Dense(128, activation="relu"),\n    layers.Dense(64, activation="relu"),\n    layers.Dense(26, activation="softmax"),\n])\nmodel.summary()\nplot_model(model,show_shapes=True, show_layer_names=True)\n'

In [15]:
def create_model(input_shape, num_classes):
    # Note: input is flipped to (height, width) instead of (width, height)
    inputs = keras.Input(shape=(input_shape[1], input_shape[0], input_shape[2]))
    
    data_augmentation = keras.Sequential(
        [
            #version tf 2.4.1: 
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
        ]
    )
    # Image augmentation block
    x = data_augmentation(inputs)
    activation_str = "elu"
    # Entry block
    #version tf 2.4.1
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    #-----------------
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation_str)(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation_str)(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation(activation_str)(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation(activation_str)(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation_str)(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

#creating model
model = create_model([DESIRED_SIZE[0], DESIRED_SIZE[1], 3], NUM_CLASSES)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 145, 218, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 145, 218, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 145, 218, 3)  0           ['sequential[0][0]']             
                                                                                                  
 conv2d (Conv2D)                (None, 73, 109, 32)  896         ['rescaling[0][0]']          

                                                                                                  
 batch_normalization_7 (BatchNo  (None, 19, 28, 512)  2048       ['separable_conv2d_5[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 10, 14, 512)  0          ['batch_normalization_7[0][0]']  
                                                                                                  
 conv2d_4 (Conv2D)              (None, 10, 14, 512)  131584      ['add_1[0][0]']                  
                                                                                                  
 add_2 (Add)                    (None, 10, 14, 512)  0           ['max_pooling2d_2[0][0]',        
                                                                  'conv2d_4[0][0]']               
          

In [16]:
#compiling model
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(LR),                    
              metrics=["accuracy"]) 

In [ ]:


#training model
'''
epochs = 20
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))
#saving trained model
model.save("trained_model_cnn.h5")

#with open('base_model.pkl','wb') as f:
#    pickle.dump(model,f)
'''

epochs = 10

for e in range(epochs):
    dataset_indexes_shuffled = np.random.permutation(np.arange(len(X_train)))
    dataset_in_batches = [dataset_indexes_shuffled[i:i+BATCH_SIZE] 
                          for i in range(0, len(dataset_indexes_shuffled), BATCH_SIZE)]
    len_dat_d4 = len(dataset_in_batches)//4
    for i, batch in enumerate(dataset_in_batches):
        loss, acc = train_on_batch(model, batch)
        
        if i in [len_dat_d4, len_dat_d4*2, len_dat_d4*3, len_dat_d4*4-1]:
            loss_v, acc_v = validate(model, X_valid)
            wandb.log({"validation_loss": loss_v, "validation_accuracy": acc_v})
            print(f"validation loss, acc: {loss_v}, {acc_v}")
        if i % 10 == 0:
            print(f"train loss, acc: {loss}, {acc}")
        if i % 50 == 0:
            model.save(f"trained_model_cnn_batch_{i}.h5")

In [ ]:
#visualize model performance
accuracy = history.history['sparse_categorical_accuracy']
val_accuracy = history.history['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 7))
plt.plot(range(epochs), accuracy, "r", label="Training Accuracy")
plt.plot(range(epochs), val_accuracy, "orange", label="Validation Accuracy")
plt.plot(range(epochs), loss, "b", label="Training Loss")
plt.plot(range(epochs), val_loss, "g", label="Validation Loss")
plt.legend(loc="lower left")
#plt.gca().set_ylim(0, 2)
plt.grid(True)

plt.show()

In [ ]:
#evaluate model
model.evaluate(X_test, y_test)

## Predicting

In [ ]:
if model not in globals():
    model = pickle.load(open('base_model.pkl', 'rb'))
    
y_proba = model.predict(X_test)
